In [134]:
import re, os
import numpy as np
import pandas as pd
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.Data import CodonTable
import scipy.stats as sc

In [135]:
files = [i for i in os.listdir('coding_genes_moralis_ref_ancient/') if 'fasta' in i]

In [136]:
dat_temp = Bio.SeqIO.to_dict(Bio.SeqIO.parse(open('coding_genes_moralis_ref_ancient/'+files[0], 'r'), 'fasta'))

In [137]:
def all_gap_or_n(seq_data):
    str_list = list(seq_data)
    count_gap = sum([i=='-' or i=='n' for i in str_list])
    return count_gap == len(str_list)

In [138]:
## Take each triplet
## Check whether there are changes at the nucleotide level
## Translate to AA, check whether they are the same
## count to a nonsynonymous or a synonymous

In [155]:
slices = [[i, i+3] for i in range(0, len(dat_temp[dat_temp.keys()[0]]), 3)]

In [196]:
def get_dnds(dat_temp):
    '''
    Note that dat_temp has to be a dictionary with two sequences only
    '''
    s1 = str(dat_temp[dat_temp.keys()[0]].seq)
    s2 = str(dat_temp[dat_temp.keys()[1]].seq)
    last_site = int(len(seq_data)/3.)*3
    s1 = s1[:last_site]
    s2 = s2[:last_site]
    if any([all_gap_or_n(i) for i in [s1, s2]]):
        return [0, 0]

    count_synonymous = 0
    count_nonsynonymous = 0
    for s in slices:
        s1_nuc = s1[s[0]:s[1]]
        s2_nuc = s2[s[0]:s[1]]
    
        if any([q in ['-', 'n', 'x', 'X'] for q in s1_nuc ]) or any([q in ['-', 'n', 'x', 'X'] for q in s2_nuc ]):
            # missing_data
            continue
        else:  
            s1_aa = Bio.Seq.translate(s1_nuc, table = 'Bacterial')
            s2_aa = Bio.Seq.translate(s2_nuc, table = 'Bacterial')
            if not(s1_aa == s2_aa):
                count_nonsynonymous = count_nonsynonymous+1
            elif not(s1_nuc == s2_nuc):
                count_synonymous = count_synonymous+1
    
    return [count_nonsynonymous, count_synonymous]

In [197]:
files = [i for i in os.listdir('coding_genes_moralis_ref_ancient/') if 'fasta' in i]

coding_genes_dnds = list()
for f in files:
    dat_temp = Bio.SeqIO.to_dict(Bio.SeqIO.parse(open('coding_genes_moralis_ref_ancient/'+f, 'r'), 'fasta'))
    dnds_temp = get_dnds(dat_temp)
    coding_genes_dnds.append([f, dnds_temp[0], dnds_temp[1]])
    out_frame = pd.DataFrame(coding_genes_dnds)

    out_frame.columns = ['gene_name', 'ns', 'ss']

    out_frame.to_csv('coding_genes_moralis_ref_ancient_dnds.csv')

In [198]:
dat_temp

{'M_oralis_MAPCycle0_ConsensusStrictN_nocontigs': SeqRecord(seq=Seq('-------------------------------------------------nnnnn...nnn', SingleLetterAlphabet()), id='M_oralis_MAPCycle0_ConsensusStrictN_nocontigs', name='M_oralis_MAPCycle0_ConsensusStrictN_nocontigs', description='M_oralis_MAPCycle0_ConsensusStrictN_nocontigs', dbxrefs=[]),
 'MethanobrevibacteroralisJMR01': SeqRecord(seq=Seq('acaaaattatctaacttattagccatatctgtattagataaatctgtacttgca...ccc', SingleLetterAlphabet()), id='MethanobrevibacteroralisJMR01', name='MethanobrevibacteroralisJMR01', description='MethanobrevibacteroralisJMR01', dbxrefs=[])}

In [190]:
get_dnds(dat_temp)

trying here


[7, 6]